In [11]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import pdb 
import time
import numpy as np
import pickle
from utils import load_data2, accuracy

if torch.cuda.is_available():
    print('cuda available')
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
    #torch.cuda.manual_seed(1)
else:
    print('cuda not available')
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor
    #torch.manual_seed(1)
    
# import files in folder util
import sys
sys.path.insert(0, 'util/')
import block 
import graph_generator as g


from sklearn.metrics import confusion_matrix

cuda not available


In [12]:
if 1 == 1:
    seed = 42
    lr = 0.01
    weight_decay = 5e-4
    hidden = 16
    dropout = 0.5
    epochs = 200
    fastmode= False
    nclass = 7

    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    # Load data: 7 classes
    adj, features, labels, idx_train, idx_val, idx_test = load_data2()
    
    cora_labels_train = labels + 1
    cora_labels_val = labels + 1
    cora_labels_test = labels + 1

    cora_labels_train[140:] = 0
    cora_labels_val[:140] = 0
    cora_labels_val[500:] = 0
    cora_labels_test[:500] = 0
    cora_labels_test[1500:] = 0
    
    adj = adj.toarray()
    adj += np.eye(adj.shape[0])
    
    cora_Estart = np.zeros((10000, 2708))
    cora_Eend = np.zeros((10000, 2708))

    # converting adjacency matrix to edge-to-start vertex matrix
    count = 0
    for i in range(adj.shape[0]):
        for j in range(adj.shape[1]):
            if adj[i,j] == 1:
                cora_Estart[count,i] = 1
                cora_Eend[count,j] = 1
                count += 1

    cora_Estart = cora_Estart[:count]
    cora_Eend = cora_Eend[:count]

Loading cora dataset...


In [13]:
# semi-supervised clustering
task_parameters = {}
task_parameters['flag_task'] = 'clustering'
task_parameters['nb_communities'] = 10
task_parameters['nb_clusters_target'] = task_parameters['nb_communities']
task_parameters['Voc'] = task_parameters['nb_communities'] + 1
task_parameters['size_min'] = 5
task_parameters['size_max'] = 25
task_parameters['p'] = 0.5
task_parameters['q'] = 0.1  
file_name = 'data/set_100_clustering_maps_p05_q01_size5_25_2017-10-31_10-25-00_.txt'
with open(file_name, 'rb') as fp:
    all_trainx = pickle.load(fp)
task_parameters['all_trainx'] = all_trainx[:100]

In [14]:
class OurConvNetcell(nn.Module):
    
    def __init__(self, dim_in, dim_out):
        super(OurConvNetcell, self).__init__()

        # conv1
        self.Ui1 = nn.Linear(dim_in, dim_out, bias=False) 
        self.Uj1 = nn.Linear(dim_in, dim_out, bias=False)
        self.Vi1 = nn.Linear(dim_in, dim_out, bias=False) 
        self.Vj1 = nn.Linear(dim_in, dim_out, bias=False)  
        self.bu1 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        self.bv1 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        
        # conv2
        self.Ui2 = nn.Linear(dim_out, dim_out, bias=False) 
        self.Uj2 = nn.Linear(dim_out, dim_out, bias=False)
        self.Vi2 = nn.Linear(dim_out, dim_out, bias=False) 
        self.Vj2 = nn.Linear(dim_out, dim_out, bias=False)  
        self.bu2 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        self.bv2 = torch.nn.Parameter( torch.FloatTensor(dim_out), requires_grad=True )
        
        # bn1, bn2
        self.bn1 = torch.nn.BatchNorm1d(dim_out)
        self.bn2 = torch.nn.BatchNorm1d(dim_out)
        
        # resnet
        self.R = nn.Linear(dim_in, dim_out, bias=False) 
            
        # init
        self.init_weights_OurConvNetcell(dim_in, dim_out, 1)
        
         
    def init_weights_OurConvNetcell(self, dim_in, dim_out, gain):
        
        # conv1
        scale = gain* np.sqrt( 2.0/ dim_in )
        self.Ui1.weight.data.uniform_(-scale, scale) 
        self.Uj1.weight.data.uniform_(-scale, scale) 
        self.Vi1.weight.data.uniform_(-scale, scale) 
        self.Vj1.weight.data.uniform_(-scale, scale) 
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.bu1.data.fill_(0)
        self.bv1.data.fill_(0)
        
        # conv2
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.Ui2.weight.data.uniform_(-scale, scale) 
        self.Uj2.weight.data.uniform_(-scale, scale) 
        self.Vi2.weight.data.uniform_(-scale, scale) 
        self.Vj2.weight.data.uniform_(-scale, scale) 
        scale = gain* np.sqrt( 2.0/ dim_out )
        self.bu2.data.fill_(0)
        self.bv2.data.fill_(0)
        
        # RN
        scale = gain* np.sqrt( 2.0/ dim_in )
        self.R.weight.data.uniform_(-scale, scale)  
            
            
    def forward(self, x, E_start, E_end):
        # E_start, E_end : E x V
        xin = x
        # conv1
        Vix = self.Vi1(x)  #  V x H_out
        Vjx = self.Vj1(x)  #  V x H_out
        x1 = torch.mm(E_end,Vix) + torch.mm(E_start,Vjx) + self.bv1  # E x H_out
        x1 = F.sigmoid(x1)
        Uix = self.Ui1(x)  #  V x H_out
        x2 = torch.mm(E_start, Uix)  #  V x H_out
        Ujx = self.Uj1(x)  #  V x H_out
        x = Uix + torch.mm(E_end.t(), x1*x2) + self.bu1 #  V x H_out
        # bn1
        x = self.bn1(x)
        # relu1
        x = F.relu(x)
        # conv2
        Vix = self.Vi2(x)  #  V x H_out
        Vjx = self.Vj2(x)  #  V x H_out
        x1 = torch.mm(E_end,Vix) + torch.mm(E_start,Vjx) + self.bv2  # E x H_out
        x1 = F.sigmoid(x1)
        Uix = self.Ui2(x)  #  V x H_out
        x2 = torch.mm(E_start, Uix)  #  V x H_out
        Ujx = self.Uj2(x)  #  V x H_out
        x = Uix + torch.mm(E_end.t(), x1*x2) + self.bu2 #  V x H_out
        # bn2
        x = self.bn2(x)
        # addition
        x = x + self.R(xin)
        # relu2
        x = F.relu(x)
            
        return x
        
        
class Graph_OurConvNet(nn.Module):
    
    def __init__(self, net_parameters):
        super(Graph_OurConvNet, self).__init__()
        
        # parameters
        flag_task = task_parameters['flag_task']
        Voc = net_parameters['Voc']
        D = net_parameters['D']
        nb_clusters_target = net_parameters['nb_clusters_target']
        H = net_parameters['H']
        L = net_parameters['L']
        
        # vector of hidden dimensions
        net_layers = []
        for layer in range(L):
            net_layers.append(H)

        # embedding
        self.encoder = nn.Embedding(Voc, D)      
        
        # CL cells
        # NOTE: Each graph convnet cell uses *TWO* convolutional operations
        net_layers_extended = [D] + net_layers # include embedding dim
        L = len(net_layers)
        list_of_gnn_cells = [] # list of NN cells
        for layer in range(L//2):
            Hin, Hout = net_layers_extended[2*layer], net_layers_extended[2*layer+2]
            list_of_gnn_cells.append(OurConvNetcell(Hin,Hout))
            
        # register the cells for pytorch
        self.gnn_cells = nn.ModuleList(list_of_gnn_cells)
              
        # fc
        Hfinal = net_layers_extended[-1]
        self.fc = nn.Linear(Hfinal,nb_clusters_target) 
        
        # init
        self.init_weights_Graph_OurConvNet(Voc,D,Hfinal,nb_clusters_target,1)
        
        # print
        print('\nnb of hidden layers=',L)
        print('dim of layers (w/ embed dim)=',net_layers_extended)      
        print('\n')
        
        # class variables
        self.L = L
        self.net_layers_extended = net_layers_extended      
        self.flag_task = flag_task
        
        
    def init_weights_Graph_OurConvNet(self, Fin_enc, Fout_enc, Fin_fc, Fout_fc, gain):

        scale = gain* np.sqrt( 2.0/ Fin_enc )
        self.encoder.weight.data.uniform_(-scale, scale)  
        scale = gain* np.sqrt( 2.0/ Fin_fc )
        self.fc.weight.data.uniform_(-scale, scale)  
        self.fc.bias.data.fill_(0)  
    
            
    def forward(self, x, E_start, E_end):
        # convnet cells  
        x = self.encoder(x) # V x D
        for layer in range(self.L//2):
            gnn_layer = self.gnn_cells[layer]            
            x = gnn_layer(x,E_start,E_end) # V x Hfinal
            
        # FC
        x = self.fc(x)
        return x
         
    def loss(self, y, y_target, weight):
        loss = nn.CrossEntropyLoss(weight=weight.type(dtypeFloat))(y,y_target)
        return loss
       
    def update(self, lr):
        update = torch.optim.Adam( self.parameters(), lr=lr )
        return update
    
    
    def update_learning_rate(self, optimizer, lr):
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        return optimizer
    
    def nb_param(self):
        return self.nb_param

In [15]:
# network parameters
net_parameters = {}
net_parameters['Voc'] = task_parameters['Voc']
net_parameters['D'] = 50
net_parameters['nb_clusters_target'] = task_parameters['nb_clusters_target']
net_parameters['H'] = 50
net_parameters['L'] = 10

net = Graph_OurConvNet(net_parameters)
if torch.cuda.is_available():
    net.cuda()
# number of network parameters
nb_param = 0
for param in net.parameters():
    nb_param += np.prod(list(param.data.size()))
print('nb_param=',nb_param,' L=',net_parameters['L'])


nb of hidden layers= 10
dim of layers (w/ embed dim)= [50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


nb_param= 115560  L= 10


In [16]:
# task parameters
flag_task = task_parameters['flag_task']
# network parameters
Voc = net_parameters['Voc']
D = net_parameters['D']
nb_clusters_target = net_parameters['nb_clusters_target']
H = net_parameters['H']
L = net_parameters['L']

# optimization parameters
learning_rate = 0.00075
max_iters = 5000
batch_iters = 100
decay_rate = 1.25

In [7]:
def get_net_datasets(train_x):
    train_y = Variable( torch.LongTensor(train_x.target).type(dtypeLong) , requires_grad=False) 
    labels = train_y.data.cpu().numpy()
    nb_classes = len(np.unique(labels)) 

    # signal
    x = train_x.signal  # V-dim
    x = Variable( torch.LongTensor(x).type(dtypeLong) , requires_grad=False)
    # graph operators -  Edge = start vertex to end vertex
    E_start = train_x.edge_to_starting_vertex # E_start = E x V mapping matrix from edge index to corresponding start vertex
    E_end   = train_x.edge_to_ending_vertex # E_end = E x V mapping matrix from edge index to corresponding end vertex
    E_start = torch.from_numpy(E_start.toarray()).type(dtypeFloat)
    E_end = torch.from_numpy(E_end.toarray()).type(dtypeFloat) 
    E_start = Variable( E_start , requires_grad=False) 
    E_end = Variable( E_end , requires_grad=False) 
    
    return x, train_y, labels, nb_classes, E_start, E_end

In [31]:
train_x = g.graph_semi_super_clu(task_parameters)
train_y = Variable( torch.LongTensor(train_x.target).type(dtypeLong) , requires_grad=False) 
labels = train_y.data.cpu().numpy()
nb_classes = len(np.unique(labels)) 

# signal
x = train_x.signal  # V-dim
x = Variable( torch.LongTensor(x).type(dtypeLong) , requires_grad=False)
# graph operators -  Edge = start vertex to end vertex
E_start = train_x.edge_to_starting_vertex.toarray() # E_start = E x V mapping matrix from edge index to corresponding start vertex
E_end   = train_x.edge_to_ending_vertex.toarray() # E_end = E x V mapping matrix from edge index to corresponding end vertex
# E_start = torch.from_numpy(E_start.toarray()).type(dtypeFloat)
# E_end = torch.from_numpy(E_end.toarray()).type(dtypeFloat) 
# E_start = Variable( E_start , requires_grad=False) 
# E_end = Variable( E_end , requires_grad=False) 



In [64]:
adj = np.zeros((len(x), len(x)))

start_node = np.argmax(E_start, 1)
end_node = np.argmax(E_end, 1)
for i in range(len(start_node)):
    # adj matrix is symmetric adj[i,j] == adj[j, i]
    adj[start_node[i], end_node[i]] = 1

In [65]:
adj

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  1.,  1.]])

In [9]:
# Optimizer
global_lr = learning_rate
global_step = 0
lr = learning_rate
optimizer = net.update(lr) 


#############
# loop over epochs
#############
t_start = time.time()
t_start_total = time.time()
average_loss_old = 1e10
running_loss = 0.0
running_total = 0
running_conf_mat = 0
running_accuracy = 0
tab_results = []
for iteration in range(max_iters):  # loop over the dataset multiple times
    train_x = g.graph_semi_super_clu(task_parameters) # gets a new graph dataset
    x, train_y, labels, nb_classes, E_start, E_end = get_net_datasets(train_x)

    # forward, loss
    y = net.forward(x, E_start, E_end)
    # compute loss weigth
    V = labels.shape[0]
    
    cluster_sizes = np.zeros(nb_classes)
    for r in range(nb_classes):
        cluster = np.where(labels==r)[0]
        cluster_sizes[r] = len(cluster)    
    weight = torch.zeros(nb_classes)
    for r in range(nb_classes):
        sumj = 0
        for j in range(nb_classes):
            if j!=r:
                sumj += cluster_sizes[j]
        weight[r] = sumj/ V 
    loss = net.loss(y,train_y,weight)
    loss_train = loss.data[0]
    running_loss += loss_train
    running_total += 1

    # confusion matrix
    S = labels
    C = np.argmax( torch.nn.Softmax(dim=0)(y).data.cpu().numpy() , axis=1)
    CM = confusion_matrix(S,C).astype(np.float32)
    nb_classes = CM.shape[0]
    #train_y = train_y.data.cpu().numpy()
    for r in range(nb_classes):
        cluster = np.where(labels==r)[0]
        CM[r,:] /= cluster.shape[0]
    running_conf_mat += CM
    running_accuracy += np.sum(np.diag(CM))/ nb_classes

    # backward, update
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # learning rate, print results
    if not iteration%batch_iters:

        # time
        t_stop = time.time() - t_start
        t_start = time.time()

        # confusion matrix
        average_conf_mat = running_conf_mat/ running_total
        running_conf_mat = 0

        # accuracy
        average_accuracy = running_accuracy/ running_total
        running_accuracy = 0

        # update learning rate 
        average_loss = running_loss/ running_total
        if average_loss > 0.99* average_loss_old:
            lr /= decay_rate
        average_loss_old = average_loss
        optimizer = net.update_learning_rate(optimizer, lr)
        running_loss = 0.0
        running_total = 0

        # save intermediate results
        tab_results.append([iteration,average_loss,100* average_accuracy,time.time()-t_start_total])

        # print results
        if 1==1:
            print('\niteration= %d, loss(%diter)= %.3f, lr= %.8f, time(%diter)= %.2f' % 
                  (iteration, batch_iters, average_loss, lr, batch_iters, t_stop))
            #print('Confusion matrix= \n', 100* average_conf_mat)
            print('accuracy= %.3f' % (100* average_accuracy))


iteration= 0, loss(100iter)= 2.455, lr= 0.00075000, time(100iter)= 0.10
accuracy= 0.500

iteration= 100, loss(100iter)= 1.874, lr= 0.00075000, time(100iter)= 12.32
accuracy= 38.302

iteration= 200, loss(100iter)= 1.049, lr= 0.00075000, time(100iter)= 11.12
accuracy= 60.524

iteration= 300, loss(100iter)= 0.774, lr= 0.00075000, time(100iter)= 11.75
accuracy= 68.504

iteration= 400, loss(100iter)= 0.685, lr= 0.00075000, time(100iter)= 11.48
accuracy= 70.827

iteration= 500, loss(100iter)= 0.672, lr= 0.00075000, time(100iter)= 11.07
accuracy= 71.677

iteration= 600, loss(100iter)= 0.624, lr= 0.00075000, time(100iter)= 11.78
accuracy= 71.788

iteration= 700, loss(100iter)= 0.640, lr= 0.00060000, time(100iter)= 11.86
accuracy= 71.154

iteration= 800, loss(100iter)= 0.614, lr= 0.00060000, time(100iter)= 12.67
accuracy= 73.760

iteration= 900, loss(100iter)= 0.571, lr= 0.00060000, time(100iter)= 13.22
accuracy= 74.712

iteration= 1000, loss(100iter)= 0.576, lr= 0.00048000, time(100iter)= 13.

In [10]:
############            
# Evaluation on 100 pre-saved data
############
running_loss = 0.0
running_total = 0
running_conf_mat = 0
running_accuracy = 0
for iteration in range(100):
    train_x = task_parameters['all_trainx'][iteration][1]
    x, train_y, labels, nb_classes, E_start, E_end = get_net_datasets(train_x)

    # forward, loss
    y = net.forward(x, E_start, E_end)
    # compute loss weigth
    V = labels.shape[0]
    cluster_sizes = np.zeros(nb_classes)
    for r in range(nb_classes):
        cluster = np.where(labels==r)[0]
        cluster_sizes[r] = len(cluster)    
    weight = torch.zeros(nb_classes)
    for r in range(nb_classes):
        sumj = 0
        for j in range(nb_classes):
            if j!=r:
                sumj += cluster_sizes[j]
        weight[r] = sumj/ V 
    loss = net.loss(y,train_y,weight)
    loss_train = loss.data[0]
    running_loss += loss_train
    running_total += 1

    # confusion matrix
    S = labels
    C = np.argmax( torch.nn.Softmax(dim=0)(y).data.cpu().numpy() , axis=1)
    CM = confusion_matrix(S,C).astype(np.float32)
    nb_classes = CM.shape[0]
    train_y = train_y.data.cpu().numpy()
    for r in range(nb_classes):
        cluster = np.where(labels==r)[0]
        CM[r,:] /= cluster.shape[0]
    running_conf_mat += CM
    running_accuracy += np.sum(np.diag(CM))/ nb_classes

    # confusion matrix
    average_conf_mat = running_conf_mat/ running_total
    average_accuracy = running_accuracy/ running_total
    average_loss = running_loss/ running_total

# print results
print('\nloss(100 pre-saved data)= %.3f, accuracy(100 pre-saved data)= %.3f' % (average_loss,100* average_accuracy))


loss(100 pre-saved data)= 0.477, accuracy(100 pre-saved data)= 76.118
